<a href="https://colab.research.google.com/github/manishmcsa/Assigment-6/blob/main/model_bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from __future__ import print_function
import batchnorm
from batchnorm import BatchNorm as BN
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
dropout_value = 0.05


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # inp Block
        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=15, kernel_size=(3, 3),
                      padding=0, bias=False),
            BN(15),
            nn.ReLU(),
            nn.Dropout2d(dropout_value)
        )  # Input_size=28*28*1 kernel_size=3*3*10 Ouput_size=26*26*10
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=15, kernel_size=(3, 3),
                      padding=0, bias=False),
            BN(15),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=26*26*10 kernel_size=3*3*20 Ouput_size=24*24*20
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=20, kernel_size=(3, 3),
                      padding=0, bias=False),
            BN(20),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=24*24*20 kernel_size=3*3*20 Ouput_size=22*22*20

        self.pool1 = nn.MaxPool2d(2,
                                  2)  # Input_size=22*22*24 Ouput_size=11*11*20

        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=15, kernel_size=(1, 1),
                      padding=0, bias=False),
            BN(15),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=11*11*20 kernel_size=1*1*10 Ouput_size=11*11*10

        self.convblock5 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=15, kernel_size=(3, 3),
                      padding=0, bias=False),
            BN(15),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=11*11*10 kernel_size=3*3*20 Ouput_size=9*9*20

        self.convblock6 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=15, kernel_size=(3, 3),
                      padding=0, bias=False),
            BN(15),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=9*9*20 kernel_size=3*3*12 Ouput_size=7*7*12

        self.convblock7 = nn.Sequential(
            nn.Conv2d(in_channels=15, out_channels=20, kernel_size=(1, 1),
                      padding=0, bias=False),
            BN(20),
            nn.ReLU(),
            nn.Dropout(dropout_value)
        )  # Input_size=7*7*12 kernel_size=1*1*10 Ouput_size=7*7*10

        self.gap = nn.Sequential(
            nn.AvgPool2d(kernel_size=7)
        )  # Input_size=7*7*10  Ouput_size=1*1*10

        self.convblock8 = nn.Sequential(
            nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(1, 1),
                      padding=0, bias=False),
            # nn.BatchNorm2d(10),
            # nn.ReLU(),
            # nn.Dropout(dropout_value)
        )  # Input_size=1*1*10  Ouput_size=1*1*10

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.gap(x)
        x = self.convblock8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [ ]:
if __name__ == '__main__':
    from torchsummary import summary

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)
    model = Net().to(device)
    summary(model, input_size=(1, 28, 28))
